In [ ]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import re
import spacy
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Load spaCy NLP model
nlp = spacy.load('en_core_web_sm')

# Function to extract email addresses
def extract_email(text):
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    emails = re.findall(email_pattern, text)
    return emails[0] if emails else None

# Function to extract phone numbers
def extract_phone_number(text):
    phone_pattern = r'\+?\d{0,2}[-.\s]?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}(?:x\d+)?'
    phone_numbers = re.findall(phone_pattern, text)

    # Clean numbers (remove brackets, dots, hyphens, spaces, and extensions)
    #cleaned_numbers = [re.sub(r"[\(\).\-x\s]", "", num) for num in phone_numbers]

    # Convert list to a single string
    return ", ".join(phone_numbers) if phone_numbers else None








# Define the name extraction function
nlp = spacy.load("en_core_web_md")

# Define the name extraction function
def extract_name(text):
    doc = nlp(text)

    # Loop through the named entities detected by spaCy
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return ent.text
    return None


# Function to extract skills

def extract_skills(text, skills_list):
    # Tokenize the input text and convert to lowercase for case-insensitive matching
    text_tokens = word_tokenize(text.lower())

    # Use a regular expression to ensure skills are matched as whole words (avoid partial matches)
    skills = [skill for skill in skills_list if re.search(r'\b' + re.escape(skill.lower()) + r'\b', ' '.join(text_tokens))]

    return skills

# Predefined list of skills (customize as needed)
skills_list = ["Python", "Java", "SQL", "Machine Learning", "NLP",
               "Data Analysis", "Excel", "C++", "JavaScript", "Tableau","AI","R","MySQL","data snalyst"]

# Function to parse resumes from a dataset
def parse_resumes(dataframe, text_column):
    parsed_data = []

    for index, row in dataframe.iterrows():
        resume_text = row[text_column]
        if pd.isnull(resume_text):  # Skip empty rows
            continue

        cleaned_text = ' '.join([word for word in resume_text.split() if word.lower() not in stopwords.words('english')])

        # Extract information
        name = extract_name(cleaned_text)
        email = extract_email(cleaned_text)
        phone_number = extract_phone_number(cleaned_text)
        skills = extract_skills(cleaned_text, skills_list)

        # Append extracted data to the list
        parsed_data.append({
            "Name": name,
            "Email": email,
            "Phone Number": phone_number,
            "Skills": ', '.join(skills)
        })

    return pd.DataFrame(parsed_data)

# Load the dataset
from google.colab import files
uploaded = files.upload()  # Upload your CSV file here

# Assuming the uploaded file is named "resume_dataset.csv"
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# Parse resumes and extract data
parsed_df = parse_resumes(df, text_column="Resume")  # Modify column name if different

# Display the parsed data
#print(parsed_df)

# Save the parsed data to a new CSV file
parsed_df.to_csv("parsed_resumes_output.csv", index=False)
print("Parsed resumes saved to 'parsed_resumes_output.csv'.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Saving sorted_resume_dataset_100.csv to sorted_resume_dataset_100.csv
Parsed resumes saved to 'parsed_resumes_output.csv'.
